In [1]:
from sympy import symbols
import sympy as sp
from sympy.physics.quantum import Operator, Operator

# --- Symbols for spectral overlap ---
eta = symbols('eta', real=True, positive=True)

# --- Symbols for polarization ---
theta = symbols('theta', real=True)
phi_1 = symbols('phi_1', real=True)
phi_2 = symbols('phi_2', real=True)


In [2]:
alpha_1 = sp.cos(theta / 2) * Operator('H', 'a', 'psi') + sp.sin(theta / 2) * sp.exp(sp.I * phi_1) * Operator('V', 'a', 'psi')
alpha_1

exp(I*phi_1)*sin(theta/2)*Operator(V,a,psi) + cos(theta/2)*Operator(H,a,psi)

In [3]:
alpha_2 = eta * (
    sp.cos(theta / 2) * Operator("H", "b", "psi")
    + sp.sin(theta / 2) * sp.exp(sp.I * phi_2) * Operator("V", "b", "psi")
) + sp.sqrt(1 - eta**2) * (
    sp.cos(theta / 2) * Operator("H", "b", "psi_perp")
    + sp.sin(theta / 2) * sp.exp(sp.I * phi_2) * Operator("V", "b", "psi_perp")
)
alpha_2 = alpha_2.expand(trig=True)
alpha_2

eta*exp(I*phi_2)*sin(theta/2)*Operator(V,b,psi) + eta*cos(theta/2)*Operator(H,b,psi) + sqrt(1 - eta**2)*exp(I*phi_2)*sin(theta/2)*Operator(V,b,psi_perp) + sqrt(1 - eta**2)*cos(theta/2)*Operator(H,b,psi_perp)

In [4]:
alpha_2.args[1]

sqrt(1 - eta**2)*cos(theta/2)*Operator(H,b,psi_perp)

In [5]:
def beam_splitter_transform(op: Operator):

    factors = op.args[:-1]
    factor = sp.prod(factors)
    pol, path, spec = op.args[-1].args

    if str(path) == 'a':
        new = (1/sp.sqrt(2)) * (Operator("c", pol, spec) + Operator("d", pol, spec))
    elif str(path) == 'b':
        new = (1/sp.sqrt(2)) * (Operator("c", pol, spec) - Operator("d", pol, spec))
    else:
        raise ValueError("Invalid path")
    terms = factor * new
    return terms

alpha_1_out_terms = [beam_splitter_transform(term) for term in alpha_1.args]
alpha_1_out = sp.expand(sum(alpha_1_out_terms), trig=True)
alpha_1_out

sqrt(2)*exp(I*phi_1)*sin(theta/2)*Operator(c,V,psi)/2 + sqrt(2)*exp(I*phi_1)*sin(theta/2)*Operator(d,V,psi)/2 + sqrt(2)*cos(theta/2)*Operator(c,H,psi)/2 + sqrt(2)*cos(theta/2)*Operator(d,H,psi)/2

In [6]:
alpha_2_out_terms = [beam_splitter_transform(term) for term in alpha_2.args]
alpha_2_out = sp.expand(sum(alpha_2_out_terms), trig=True)
alpha_2_out

sqrt(2)*eta*exp(I*phi_2)*sin(theta/2)*Operator(c,V,psi)/2 - sqrt(2)*eta*exp(I*phi_2)*sin(theta/2)*Operator(d,V,psi)/2 + sqrt(2)*eta*cos(theta/2)*Operator(c,H,psi)/2 - sqrt(2)*eta*cos(theta/2)*Operator(d,H,psi)/2 + sqrt(2)*sqrt(1 - eta**2)*exp(I*phi_2)*sin(theta/2)*Operator(c,V,psi_perp)/2 - sqrt(2)*sqrt(1 - eta**2)*exp(I*phi_2)*sin(theta/2)*Operator(d,V,psi_perp)/2 + sqrt(2)*sqrt(1 - eta**2)*cos(theta/2)*Operator(c,H,psi_perp)/2 - sqrt(2)*sqrt(1 - eta**2)*cos(theta/2)*Operator(d,H,psi_perp)/2

In [7]:
bs_out = alpha_1_out * alpha_2_out
bs_out = sp.simplify(bs_out)
bs_out = sp.expand(bs_out, trig=True)
bs_out

-eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)*Operator(d,V,psi)/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)**2/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)*Operator(c,V,psi)/2 - eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)**2/2 + eta*exp(I*phi_1)*sin(theta/2)*cos(theta/2)*Operator(c,V,psi)*Operator(c,H,psi)/2 - eta*exp(I*phi_1)*sin(theta/2)*cos(theta/2)*Operator(c,V,psi)*Operator(d,H,psi)/2 + eta*exp(I*phi_1)*sin(theta/2)*cos(theta/2)*Operator(d,V,psi)*Operator(c,H,psi)/2 - eta*exp(I*phi_1)*sin(theta/2)*cos(theta/2)*Operator(d,V,psi)*Operator(d,H,psi)/2 + eta*exp(I*phi_2)*sin(theta/2)*cos(theta/2)*Operator(c,H,psi)*Operator(c,V,psi)/2 - eta*exp(I*phi_2)*sin(theta/2)*cos(theta/2)*Operator(c,H,psi)*Operator(d,V,psi)/2 + eta*exp(I*phi_2)*sin(theta/2)*cos(theta/2)*Operator(d,H,psi)*Operator(c,V,psi)/2 - eta*exp(I*phi_2)*sin(theta/2)*cos(theta/2)*Operator(d,H,psi)*Operator(d,V,psi)/2 - eta*cos(theta/2)**2*O

In [62]:
# trace out the psi/psi_perp states
bs_out = bs_out.subs(Operator('psi', 'a', 'psi'), 0)
bs_out = bs_out.subs(Operator('psi_perp', 'b', 'psi_perp'), 0)
bs_out = sp.simplify(bs_out)
bs_out = sp.expand(bs_out, trig=True)
bs_out

-eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)*Operator(d,V,psi)/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)**2/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)*Operator(c,V,psi)/2 - eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)**2/2 + eta*exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(c,H,psi)/4 - eta*exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(d,H,psi)/4 + eta*exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(c,H,psi)/4 - eta*exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(d,H,psi)/4 + eta*exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(c,V,psi)/4 - eta*exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(d,V,psi)/4 + eta*exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(c,V,psi)/4 - eta*exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(d,V,psi)/4 - eta*cos(theta/2)**2*Operator(c,H,psi)*Operator(d,H,psi)/2 + eta*cos(theta/2)**2*Operator(c,H,psi)**2/2 + eta*cos(theta/2)**2*Operator(d,H,psi

# Trace out psi

In [80]:
# To trace out the psi states, we can use the fact that
# <psi|psi> = 1 and <psi_perp|psi_perp> = 1, and all other inner products are zero.


bs_out_partial_traced_terms = []
for i, term in enumerate(bs_out.args):
    operators = [o for o in term.args if isinstance(o, Operator)]
    prod = sp.prod([o for o in term.args if not isinstance(o, Operator)])
    if len(operators) == 2:
        if operators[0].args[-1] == operators[1].args[-1]:
            bs_out_partial_traced_terms.append(term)
    else:
        bs_out_partial_traced_terms.append(term)
        
bs_out_partial_traced = sp.expand(sum(bs_out_partial_traced_terms), trig=True)
bs_out_partial_traced



-eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)*Operator(d,V,psi)/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)**2/2 + eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)*Operator(c,V,psi)/2 - eta*exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)**2/2 + eta*exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(c,H,psi)/4 - eta*exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(d,H,psi)/4 + eta*exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(c,H,psi)/4 - eta*exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(d,H,psi)/4 + eta*exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(c,V,psi)/4 - eta*exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(d,V,psi)/4 + eta*exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(c,V,psi)/4 - eta*exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(d,V,psi)/4 - eta*cos(theta/2)**2*Operator(c,H,psi)*Operator(d,H,psi)/2 + eta*cos(theta/2)**2*Operator(c,H,psi)**2/2 + eta*cos(theta/2)**2*Operator(d,H,psi

In [ ]:
# ensure it is a valid operator
bs_out_partial_traced = sp.simplify(bs_out_partial_traced)
bs_out_partial_traced = sp.expand(bs_out_partial_traced, trig=True)
bs_out_partial_traced = bs_out_partial_traced.subs(Operator('psi', 'a', 'psi'), 0)
bs_out_partial_traced = bs_out_partial_traced.subs(Operator('psi_perp', 'b', 'psi_perp'), 0)
bs_out_partial_traced = sp.simplify(bs_out_partial_traced)
bs_out_partial_traced = sp.expand(bs_out_partial_traced, trig=True)
bs_out_partial_traced

# Relevant projectors

In [114]:
# get the sum of the coefficients of the following terms
H1H1 = Operator("c", "H", "psi") * Operator("c", "H", "psi")
H2H2 = Operator("d", "H", "psi") * Operator("d", "H", "psi")

V1V1 = Operator("c", "V", "psi") * Operator("c", "V", "psi")
V2V2 = Operator("d", "V", "psi") * Operator("d", "V", "psi")

H1V1 = Operator("c", "H", "psi") * Operator("c", "V", "psi")
H2V2 = Operator("d", "H", "psi") * Operator("d", "V", "psi")

H1H2 = Operator("c", "H", "psi") * Operator("d", "H", "psi")
H2H1 = Operator("d", "H", "psi") * Operator("c", "H", "psi")

V1V2 = Operator("c", "V", "psi") * Operator("d", "V", "psi")
V2V1 = Operator("d", "V", "psi") * Operator("c", "V", "psi")

H1V2 = Operator("c", "H", "psi") * Operator("d", "V", "psi")
V2H1 = Operator("d", "V", "psi") * Operator("c", "H", "psi")

V1H2 = Operator("c", "V", "psi") * Operator("d", "H", "psi")
H2V1 = Operator("d", "H", "psi") * Operator("c", "V", "psi")

H1H1

Operator(c,H,psi)**2

# With eta = 1
Should recover the original probabilities

In [98]:
# try for eta = 1
bs_out_eta_1 = bs_out_partial_traced.subs(eta, 1)
bs_out_eta_1 = sp.simplify(bs_out_eta_1)
bs_out_eta_1 = sp.expand(bs_out_eta_1, trig=True)
bs_out_eta_1

-exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)*Operator(d,V,psi)/2 + exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(c,V,psi)**2/2 + exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)*Operator(c,V,psi)/2 - exp(I*phi_1)*exp(I*phi_2)*sin(theta/2)**2*Operator(d,V,psi)**2/2 + exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(c,H,psi)/4 - exp(I*phi_1)*sin(theta)*Operator(c,V,psi)*Operator(d,H,psi)/4 + exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(c,H,psi)/4 - exp(I*phi_1)*sin(theta)*Operator(d,V,psi)*Operator(d,H,psi)/4 + exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(c,V,psi)/4 - exp(I*phi_2)*sin(theta)*Operator(c,H,psi)*Operator(d,V,psi)/4 + exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(c,V,psi)/4 - exp(I*phi_2)*sin(theta)*Operator(d,H,psi)*Operator(d,V,psi)/4 - cos(theta/2)**2*Operator(c,H,psi)*Operator(d,H,psi)/2 + cos(theta/2)**2*Operator(c,H,psi)**2/2 + cos(theta/2)**2*Operator(d,H,psi)*Operator(c,H,psi)/2 - cos(theta/2)**2*Operator(d,H,psi)**2

In [119]:
coeff_H1H1 = sp.expand(bs_out_eta_1.coeff(H1H1) **2, trig=True)

sp.expand(coeff_H1H1, trig=True)
coeff_H1H1

cos(theta/2)**4/4

In [116]:
coeff_V1V2 = bs_out_eta_1.coeff(V1V2) + bs_out_eta_1.coeff(V2V1)
p_V1V2 = sp.expand(coeff_V1V2 * coeff_V1V2.adjoint(), trig=True)

p_V1V2

0

In [ ]:
p_V1V1 = sp.expand(bs_out_eta_1.coeff(V1V1) ** 2, trig=True)